In [1]:
import numpy as np
import pandas as pd

In [2]:
nb_svm_wt = 2/float(9)
glove_100_wt = 3/float(9)
custom_100_wt = 4/float(9)

In [21]:
#nb_svm = pd.read_csv('submission/test/NBSVM_base_submission.csv')
#glove_100 = pd.read_csv('submission/test/LSTM-base_100-submission2.csv')
custom_100 = pd.read_csv('submission/LSTM-base-submission1.csv')
tfidf = pd.read_csv('submission/word-character-n-grams-tfidf-regressions-lb-051.csv')
xgboost = pd.read_csv('submission/base_xgboost.csv')

In [24]:
custom_tfid_join = custom_100.join(xgboost, rsuffix='_x')
full_join = custom_tfid_join.join(tfidf, rsuffix='_t')
full_join.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,id_x,toxic_x,severe_toxic_x,...,threat_x,insult_x,identity_hate_x,id_t,toxic_t,severe_toxic_t,obscene_t,threat_t,insult_t,identity_hate_t
0,00001cee341fdb12,0.997376,3.421685e-01,9.693919e-01,9.319638e-02,7.700840e-01,3.498317e-01,00001cee341fdb12,0.995758,0.301491,...,0.026276,0.954673,0.145898,00001cee341fdb12,0.999850,0.166629,0.999582,0.065705,0.991182,0.478361
1,0000247867823ef7,0.000019,3.273130e-12,7.594855e-07,2.138816e-10,9.456933e-07,1.467620e-08,0000247867823ef7,0.063557,0.005738,...,0.001615,0.026641,0.005501,0000247867823ef7,0.008019,0.004266,0.006445,0.000499,0.004568,0.002605
2,00013b17ad220c46,0.003472,1.558526e-05,5.973944e-03,3.523753e-05,2.155448e-03,2.834277e-04,00013b17ad220c46,0.086429,0.005869,...,0.002406,0.027423,0.004752,00013b17ad220c46,0.025230,0.008034,0.016308,0.001218,0.006552,0.002247
3,00017563c3f7919a,0.000032,8.953008e-12,1.607935e-06,1.248840e-08,6.508925e-07,1.543622e-08,00017563c3f7919a,0.019357,0.000479,...,0.000595,0.008343,0.001452,00017563c3f7919a,0.002481,0.001001,0.001842,0.000697,0.002761,0.000383
4,00017695ad8997eb,0.000145,3.322608e-08,6.544872e-05,9.616349e-06,3.016094e-05,1.070794e-06,00017695ad8997eb,0.081306,0.003094,...,0.000888,0.028416,0.003410,00017695ad8997eb,0.018513,0.001046,0.004600,0.000557,0.008488,0.000902


In [25]:
for col in custom_100.columns:
    if col != 'id':
        full_join[col] = 0.25 * full_join[col+'_x'] + 0.40 * full_join[col] + 0.35 * full_join[col+'_t']
        full_join[col] = full_join[col].clip(0+1e12, 1-1e12)

full_join[custom_100.columns].to_csv('submission/ensemble_full_25_35_40.csv', index=False)


In [5]:
nb_glove_join = nb_svm.join(glove_100, rsuffix='_g')
full_join = nb_glove_join.join(custom_100, rsuffix='_c')

In [6]:
full_join.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,id_g,toxic_g,severe_toxic_g,...,threat_g,insult_g,identity_hate_g,id_c,toxic_c,severe_toxic_c,obscene_c,threat_c,insult_c,identity_hate_c
0,00001cee341fdb12,0.999978,0.032485,0.999972,0.002410,0.959821,0.109278,00001cee341fdb12,0.992377,2.129427e-01,...,1.605575e-02,0.878010,3.616224e-02,00001cee341fdb12,0.997376,3.421685e-01,9.693919e-01,9.319638e-02,7.700840e-01,3.498317e-01
1,0000247867823ef7,0.002732,0.000599,0.001889,0.000091,0.001990,0.000414,0000247867823ef7,0.000073,2.720313e-09,...,4.752737e-09,0.000010,2.486503e-07,0000247867823ef7,0.000019,3.273130e-12,7.594855e-07,2.138816e-10,9.456933e-07,1.467620e-08
2,00013b17ad220c46,0.008361,0.000657,0.003959,0.000087,0.002514,0.000307,00013b17ad220c46,0.002451,2.784698e-06,...,1.380072e-06,0.000310,2.899905e-04,00013b17ad220c46,0.003472,1.558526e-05,5.973944e-03,3.523753e-05,2.155448e-03,2.834277e-04
3,00017563c3f7919a,0.000935,0.000240,0.001209,0.000164,0.001093,0.000317,00017563c3f7919a,0.000119,1.284240e-09,...,3.575138e-08,0.000024,5.073461e-08,00017563c3f7919a,0.000032,8.953008e-12,1.607935e-06,1.248840e-08,6.508925e-07,1.543622e-08
4,00017695ad8997eb,0.009646,0.000471,0.002096,0.000126,0.002451,0.000361,00017695ad8997eb,0.000084,1.269676e-08,...,1.069603e-07,0.000005,5.379580e-07,00017695ad8997eb,0.000145,3.322608e-08,6.544872e-05,9.616349e-06,3.016094e-05,1.070794e-06


In [7]:
nb_svm.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999978,0.032485,0.999972,0.002410,0.959821,0.109278
1,0000247867823ef7,0.002732,0.000599,0.001889,0.000091,0.001990,0.000414
2,00013b17ad220c46,0.008361,0.000657,0.003959,0.000087,0.002514,0.000307
3,00017563c3f7919a,0.000935,0.000240,0.001209,0.000164,0.001093,0.000317
4,00017695ad8997eb,0.009646,0.000471,0.002096,0.000126,0.002451,0.000361


In [8]:
ensemble = pd.DataFrame(columns=nb_svm.columns)
ensemble.columns

Index([u'id', u'toxic', u'severe_toxic', u'obscene', u'threat', u'insult',
       u'identity_hate'],
      dtype='object')

In [9]:
ensemble.id = full_join.id
ensemble.toxic = (nb_svm_wt * full_join.toxic + custom_100_wt * full_join.toxic_c + glove_100_wt * full_join.toxic_g)/3
ensemble.severe_toxic = (nb_svm_wt * full_join.severe_toxic + custom_100_wt * full_join.severe_toxic_c + glove_100_wt * full_join.severe_toxic_g)/3
ensemble.obscene = (nb_svm_wt * full_join.obscene + custom_100_wt * full_join.obscene_c + glove_100_wt * full_join.obscene_g)/3
ensemble.threat = (nb_svm_wt * full_join.threat + custom_100_wt * full_join.threat_c + glove_100_wt * full_join.threat_g)/3
ensemble.insult = (nb_svm_wt * full_join.insult + custom_100_wt * full_join.insult_c + glove_100_wt * full_join.insult_g)/3
ensemble.identity_hate = (nb_svm_wt * full_join.identity_hate + custom_100_wt * full_join.identity_hate_c + glove_100_wt * full_join.identity_hate_g)/3

In [10]:
ensemble.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.332096,0.076758,0.322080,0.015769,0.282741,0.063940
1,0000247867823ef7,0.000213,0.000044,0.000142,0.000007,0.000149,0.000031
2,00013b17ad220c46,0.001406,0.000051,0.001211,0.000012,0.000540,0.000097
3,00017563c3f7919a,0.000087,0.000018,0.000093,0.000012,0.000084,0.000023
4,00017695ad8997eb,0.000745,0.000035,0.000167,0.000011,0.000187,0.000027


In [11]:
ensemble.to_csv('submission/ensemble_diff_weights.csv', index=False)

In [13]:
custom_100.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997376,3.421685e-01,9.693919e-01,9.319638e-02,7.700840e-01,3.498317e-01
1,0000247867823ef7,0.000019,3.273130e-12,7.594855e-07,2.138816e-10,9.456933e-07,1.467620e-08
2,00013b17ad220c46,0.003472,1.558526e-05,5.973944e-03,3.523753e-05,2.155448e-03,2.834277e-04
3,00017563c3f7919a,0.000032,8.953008e-12,1.607935e-06,1.248840e-08,6.508925e-07,1.543622e-08
4,00017695ad8997eb,0.000145,3.322608e-08,6.544872e-05,9.616349e-06,3.016094e-05,1.070794e-06


In [14]:
custom_100.columns

Index([u'id', u'toxic', u'severe_toxic', u'obscene', u'threat', u'insult',
       u'identity_hate'],
      dtype='object')

In [18]:
for col in custom_100.columns:
    if col != 'id':
        custom_100[col] = custom_100[col].clip(0+1e12, 1-1e12)

In [19]:
custom_100.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997376,3.421685e-01,9.693919e-01,9.319638e-02,7.700840e-01,3.498317e-01
1,0000247867823ef7,0.000019,3.273130e-12,7.594855e-07,2.138816e-10,9.456933e-07,1.467620e-08
2,00013b17ad220c46,0.003472,1.558526e-05,5.973944e-03,3.523753e-05,2.155448e-03,2.834277e-04
3,00017563c3f7919a,0.000032,8.953008e-12,1.607935e-06,1.248840e-08,6.508925e-07,1.543622e-08
4,00017695ad8997eb,0.000145,3.322608e-08,6.544872e-05,9.616349e-06,3.016094e-05,1.070794e-06


In [23]:
pd.DataFrame.clip?